In [50]:
import pathlib

architecture_result_paths = list(pathlib.Path("../results/gorillas").glob("*"))
print("Number of architectures:", len(list(architecture_result_paths)))

Number of architectures: 14


In [51]:
import json
import pandas as pd
import statistics

tials_that_should_exist = []
for architecture_result_path in architecture_result_paths:
    tials_that_should_exist.append(architecture_result_path.name)
    tials_that_should_exist.append(architecture_result_path.name + "-QAT")
    tials_that_should_exist.append(architecture_result_path.name + "-PTQ")
    tials_that_should_exist.append(architecture_result_path.name + "-QAT&KD")
    

dataframe_list = []
trials_that_exist = []
for architecture_result_path in architecture_result_paths:
    trials = list(architecture_result_path.glob("*"))
    
    for trial in trials:
        config_dict = json.loads((trial / "config.json").read_text())
        metrics_dict = json.loads((trial / "metrics.json").read_text())
        
        name = trial.parent.name
        if "use_quantization_aware_training" not in config_dict.keys():
            continue
        
        if config_dict["use_quantization_aware_training"]["0"]:
            name += "-QAT"
        if config_dict["loss_mode"]["0"] == "distillation/offline/response-based":
            name += "&KD"
            
        if "aggregated/cxlkfold/val/embeddings/knn/accuracy_max" not in metrics_dict.keys():
            continue
        else:
            
            important_metrics = [
                'val/embeddings/knn/accuracy_max', "val/embeddings/knn/precision_max", "val/embeddings/knn/f1_max", "val/embeddings/knn5/accuracy_max", "val/embeddings/knn5/precision_max", "val/embeddings/knn5/f1_max", "val/embeddings/knn5/accuracy_top5_max",
                'val/embeddings/knn_crossvideo/accuracy_max', "val/embeddings/knn_crossvideo/precision_max", "val/embeddings/knn_crossvideo/f1_max", "val/embeddings/knn5_crossvideo/accuracy_max", "val/embeddings/knn5_crossvideo/precision_max", "val/embeddings/knn5_crossvideo/f1_max", "val/embeddings/knn5_crossvideo/accuracy_top5_max",
            ]
            dataframe_entry = {"name": name}
            for metric in important_metrics:
                values = []
                metric_name = metric.replace("knn/","knn-1/").replace('knn_', "knn-1_").split("/")[-2].replace("knn5", "knn-5")+ " " + metric.split("/")[-1].replace("_max", "")
                for fold in range(5):
                    values.append(metrics_dict[f"cxlkfold/fold-{fold}/" + metric]['0'])
                
                    
                dataframe_entry[metric_name + " stdev"] = statistics.stdev(values)
                dataframe_entry[metric_name + " mean"] = statistics.mean(values)
                dataframe_entry[metric_name + " min"] = min(values)
                dataframe_entry[metric_name + " max"] = max(values)
                dataframe_entry[metric_name] = metrics_dict["aggregated/cxlkfold/" + metric]["0"]
                    
            dataframe_list.append(dataframe_entry)
            trials_that_exist.append(name)
            
dataframe = pd.DataFrame(dataframe_list)
dataframe.to_csv("gorillas.csv", index=False)
    
        

In [52]:
# get data from ptq

import pathlib
import pandas as pd
import json
import statistics

architecture_result_paths = list(pathlib.Path("../runs").glob("*"))
print("Number of architectures:", len(list(architecture_result_paths)))

for architecture_path in architecture_result_paths:
    results = json.loads((architecture_path / "results.json").read_text())
    
    dataframe_entry = {"name": architecture_path.name + "-PTQ"}
    for metric in important_metrics:
        metric_name = metric.replace("knn/","knn-1/").replace('knn_', "knn-1_").split("/")[-2].replace("knn5", "knn-5")+ " " + metric.split("/")[-1].replace("_max", "")
        knn = metric.replace("knn/", "knn1/").replace('knn_', "knn1_").split("/")[-2]
        metric_name_2 = metric.split("/")[-1].replace("_max", "")
        
        values = []
        for fold in range(5):
            try:
                values.append(results[f"quantized-fold-{fold}"][knn][metric_name_2] )
            except Exception as e:
                print(f"Error with {architecture_path.name} {knn} {metric_name_2}")
                values.append(0)
            
        dataframe_entry[metric_name + " stdev"] = statistics.stdev(values)
        dataframe_entry[metric_name + " mean"] = statistics.mean(values)
        dataframe_entry[metric_name + " min"] = min(values)
        dataframe_entry[metric_name + " max"] = max(values)
        dataframe_entry[metric_name] = min(values)
        
    print(dataframe_entry)
            
    dataframe_list.append(dataframe_entry)
    trials_that_exist.append(architecture_path.name + "-PTQ")
    
dataframe = pd.DataFrame(dataframe_list)
dataframe.to_csv("gorillas.csv", index=False)

Number of architectures: 14
{'name': 'Embedding-ResNet-152-CXL-OpenSet-PTQ', 'knn-1 accuracy stdev': 0.0363329066074247, 'knn-1 accuracy mean': 0.74238493442, 'knn-1 accuracy min': 0.6875000596, 'knn-1 accuracy max': 0.7866666317, 'knn-1 accuracy': 0.6875000596, 'knn-1 precision stdev': 0.030594921269803305, 'knn-1 precision mean': 0.73274403812, 'knn-1 precision min': 0.6864588261, 'knn-1 precision max': 0.7629253864, 'knn-1 precision': 0.6864588261, 'knn-1 f1 stdev': 0.035731853917384766, 'knn-1 f1 mean': 0.72862380742, 'knn-1 f1 min': 0.6766055822, 'knn-1 f1 max': 0.7654221058, 'knn-1 f1': 0.6766055822, 'knn-5 accuracy stdev': 0.03597912736612114, 'knn-5 accuracy mean': 0.6969523668000001, 'knn-5 accuracy min': 0.65625, 'knn-5 accuracy max': 0.7366071343, 'knn-5 accuracy': 0.65625, 'knn-5 precision stdev': 0.05241649274305288, 'knn-5 precision mean': 0.62029548884, 'knn-5 precision min': 0.5641084909, 'knn-5 precision max': 0.6811719537, 'knn-5 precision': 0.5641084909, 'knn-5 f1 st

In [53]:
from math import sqrt
import matplotlib 

SPINE_COLOR = 'gray'

def latexify(fig_width=None, fig_height=None, columns=1):
    """Set up matplotlib's RC params for LaTeX plotting.
    Call this before plotting a figure.

    Parameters
    ----------
    fig_width : float, optional, inches
    fig_height : float,  optional, inches
    columns : {1, 2}
    """

    # code adapted from http://www.scipy.org/Cookbook/Matplotlib/LaTeX_Examples

    # Width and max height in inches for IEEE journals taken from
    # computer.org/cms/Computer.org/Journal%20templates/transactions_art_guide.pdf

    assert(columns in [1,2])

    if fig_width is None:
        fig_width = 3.39 if columns==1 else 6.9 # width in inches
        #fig_width = 12 if columns==1 else 17 # width in inches

    if fig_height is None:
        golden_mean = (sqrt(5)-1.0)/2.0    # Aesthetic ratio
        fig_height = fig_width*golden_mean # height in inches

    MAX_HEIGHT_INCHES = 8.0
    if fig_height > MAX_HEIGHT_INCHES:
        print("WARNING: fig_height too large:" + fig_height + 
              "so will reduce to" + MAX_HEIGHT_INCHES + "inches.")
        fig_height = MAX_HEIGHT_INCHES

    params = {'backend': 'ps',
              'text.latex.preamble': '\\usepackage{gensymb}',
              'axes.labelsize': 8, # fontsize for x and y labels (was 10)
              'axes.titlesize': 8,
              'lines.linewidth': 0.5,
              'axes.linewidth': 0.5,
              #'text.fontsize': 8, # was 10
              'legend.fontsize': 8, # was 10
              'xtick.labelsize': 8,
              'ytick.labelsize': 8,
              'lines.markersize': 2,
              'text.usetex': True,
              'figure.figsize': [fig_width,fig_height],
              'font.family': 'serif'
    }

    matplotlib.rcParams.update(params)


def format_axes(ax):

    for spine in ['top', 'right']:
        ax.spines[spine].set_visible(False)

    for spine in ['left', 'bottom']:
        ax.spines[spine].set_color(SPINE_COLOR)
        ax.spines[spine].set_linewidth(0.5)

    ax.xaxis.set_ticks_position('bottom')
    ax.yaxis.set_ticks_position('left')

    for axis in [ax.xaxis, ax.yaxis]:
        axis.set_tick_params(direction='out', color=SPINE_COLOR)

    return ax

latexify()



In [54]:
tails_that_are_missing = set(tials_that_should_exist) - set(trials_that_exist)
tails_that_are_missing = sorted(list(tails_that_are_missing))

missing_kd = []
missing_baseline = []
missing_qat = []
missing_ptq = []
for tail in tails_that_are_missing:
    if "&KD" in tail:
        missing_kd.append(tail)
    elif "-QAT" in tail:
        missing_qat.append(tail)
    elif "-PTQ" in tail:
        missing_ptq.append(tail)
    else:
        missing_baseline.append(tail)
        
with open("missing.txt", "w") as file:
    file.write("Missing KD:\n")
    for tail in missing_kd:
        file.write(tail + "\n")
        
    file.write("\nMissing QAT:\n")
    for tail in missing_qat:
        file.write(tail + "\n")
        
    file.write("\nMissing Baseline:\n")
    for tail in missing_baseline:
        file.write(tail + "\n") 
        
    file.write("\nMissing PTQ:\n")
    for tail in missing_ptq:
        file.write(tail + "\n")

In [55]:
import pandas as pd
import matplotlib.pyplot as plt
import re

from matplotlib.container import ErrorbarContainer

# Function to extract architecture and variation type, removing "Embedding-" prefix
def extract_architecture_and_type(name):
    base_name = re.sub(r'^Embedding-', '', name)
    base_name = re.sub(r'-QAT&KD|-QAT|-PTQ|-CXL-OpenSet|-CXL-Open', '', base_name)
    
    if 'PTQ' in name:
        var_type = 'PTQ'
    elif 'QAT&KD' in name:
        var_type = 'QAT\&KD'
    elif 'QAT' in name:
        var_type = 'QAT'
    else:
        var_type = 'Baseline'
        
    return base_name, var_type

# Function to annotate bars with relative changes
def annotate_bars(ax, grouped_df, include_variations):
    prev_heights = [0] * len(grouped_df)
    
    new_ax = [container for container in ax.containers if not isinstance(container, ErrorbarContainer)]
    
    for i, container in enumerate(new_ax):
        new_prev_heights = []
        variation = include_variations[i]
        
        
        for j, bar in enumerate(container):
            y = bar.get_height() + 0.04
            x = bar.get_x() + bar.get_width() / 2
            new_prev_heights.append(y)
            
            if variation != 'Baseline' and 'Baseline' in grouped_df.columns:
                baseline = grouped_df.iloc[j]['Baseline']
                if baseline != 0:  # To avoid division by zero
                    change = ((y - baseline) / baseline) * 100
                    ax.annotate(f'{change:+.2f}%', (x, y), ha='center', va='bottom', color='black', fontweight='bold')
        
        prev_heights = new_prev_heights if variation != 'Baseline' else prev_heights

# Function to plot bar charts with error bars
def plot_accuracy_bars_with_error(grouped_df, grouped_df_stdev, variations, title):
    fig, ax = plt.subplots(figsize=(14, 10))
    
    
    # Plot the bar chart with error bars
    grouped_df[variations].plot(kind='bar', ax=ax, yerr=grouped_df_stdev[variations], capsize=2)
    
    # Annotate the bars
    # annotate_bars(ax, grouped_df, variations)
    
    title.replace("&", "\&")
    # Set title and labels
    plt.title(title)
    plt.ylabel('KNN-1 Accuracy')
    plt.xlabel('Architecture')
    plt.xticks(rotation=45, ha='right')
    plt.legend(title='Variation')
    # plt.tight_layout()
    plt.show()
    plt.savefig(title + ".pdf")

# Main function to process data and generate plots
def main(df):
    # Extract architecture and variation
    df['Architecture'], df['Variation'] = zip(*df['name'].apply(extract_architecture_and_type))

    # Order by model size
    model_size_order = ['Small', 'Medium', 'Base', 'Large', '18', '50', '152']
    arch_order = sorted(
        df['Architecture'].unique(), 
        key=lambda x: (
            re.sub(r'(Small|Medium|Base|Large|18|50|152)', '', x), 
            model_size_order.index(re.search(r'(Small|Medium|Base|Large|18|50|152)', x).group(0)) if re.search(r'(Small|Medium|Base|Large|18|50|152)', x) else -1
        )
    )

    # Group by architecture and variation, taking max accuracy and standard deviation
    grouped_df = df.groupby(['Architecture', 'Variation'])['knn-1 accuracy'].max().unstack()
    grouped_df_stdev = df.groupby(['Architecture', 'Variation'])['knn-1 accuracy stdev'].max().unstack()

    # Reorder based on architecture order
    grouped_df = grouped_df.loc[arch_order]
    grouped_df_stdev = grouped_df_stdev.loc[arch_order]
    
    grouped_df_crossvideo = df.groupby(['Architecture', 'Variation'])['knn-1_crossvideo accuracy'].max().unstack()
    grouped_df_stdev_crossvideo = df.groupby(['Architecture', 'Variation'])['knn-1_crossvideo accuracy stdev'].max().unstack()
    
    grouped_df_crossvideo = grouped_df_crossvideo.loc[arch_order]
    grouped_df_stdev_crossvideo = grouped_df_stdev_crossvideo.loc[arch_order]

    # Plot the graphs
    # plot_accuracy_bars_with_error(grouped_df, grouped_df_stdev, ['Baseline', 'PTQ'], 'KNN-1 Accuracy for Each Architecture (Baseline vs PTQ)')
    # plot_accuracy_bars_with_error(grouped_df, grouped_df_stdev, ['Baseline', 'PTQ', 'QAT'], 'KNN-1 Accuracy for Each Architecture (Baseline, PTQ, and QAT)')
    plot_accuracy_bars_with_error(grouped_df, grouped_df_stdev, ['Baseline', 'PTQ', 'QAT', 'QAT\&KD'], 'KNN-1 Accuracy for Each Architecture (Baseline, PTQ, QAT, and QAT KD)')
    plot_accuracy_bars_with_error(grouped_df_crossvideo, grouped_df_stdev_crossvideo, ['Baseline', 'PTQ', 'QAT', 'QAT\&KD'], 'KNN-1 Crossvideo Accuracy for Each Architecture (Baseline, PTQ, QAT, and QAT KD)')

# Call the main function with your DataFrame
df = dataframe
main(df)


/tmp/ipykernel_1327708/3720265708.py:49: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig, ax = plt.subplots(figsize=(14, 10))
